In [2]:
## RECSYS CHALLENGE 2020 ##
# Evaluation or scoring?
eval = True

In [1]:
# Loading from CSV files...
from RecLib.DataLoad import *
UCM_age, ICM_subclass, ICM_asset, ICM_price, UCM_region, target_users, URM = dataLoad()

In [ ]:
import scipy.sparse as sps
URM = URM.tocsr()

In [ ]:
# Split dataset (train % of .9999 gives similar performance on test set and competition set)
from Notebooks_utils.data_splitter import train_test_holdout
URM_train, URM_test = train_test_holdout(URM, train_perc = 0.8)


if not eval:
    URM_train = URM
else:
    from Base.Evaluation.Evaluator import EvaluatorHoldout
    from RecLib.Evaluate import *
    evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
# Load Item & User Content matrix
ICM, UCM = contentMatrixLoad(URM_train, ICM_subclass, ICM_price, ICM_asset, UCM_age, UCM_region)


In [ ]:
# SLIM BPR Recommender
if True:
    from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython 
    slim_rec = SLIM_BPR_Cython(URM_train, recompile_cython=False, verbose = False)
    MAP_LIST = []
    epochsList = [300]
    batchSize = [50]
    tklist = [10]
    lrs = [1e-3]

    for epochsN in epochsList:
        for bs in batchSize:
            for tk in tklist:
                for lr in lrs:
                    do_not_display_hystory = slim_rec.fit(epochs=epochsN, batch_size=bs, sgd_mode='adagrad', learning_rate=lr, topK = tk)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(slim_rec))[0][10]
                        MAP_LIST.append(('epoch, batch, topK, lr :' + str(epochsN) + ' ' + str(bs)+ ' ' + str(tk) + ' ' + str(lr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

In [ ]:
# ItemKNN Content Based Filtering
if True:
    from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender 
    itemKNNCBF = ItemKNNCBFRecommender(URM_train, ICM)
    MAP_LIST = []
    tklist = [3]
    shrinklist = [10]

    for tk in tklist:
        for sr in shrinklist:
            itemKNNCBF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(itemKNNCBF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

In [1]:
#%%

if True:
    from MatrixFactorization.PureSVDRecommender import PureSVDRecommender 
    pureSVD = PureSVDRecommender(URM_train)
    MAP_LIST = []
    nfactorlist = [400]

    for n in nfactorlist:
        pureSVD.fit(num_factors=n, random_seed = None)
        if eval:
            dict_scores = (evaluator_validation.evaluateRecommender(pureSVD))[0][10]
            MAP_LIST.append(('num factors :' + str(n) , dict_scores['MAP']))
    if eval:
        print(MAP_LIST)  

NameError: name 'URM_train' is not defined

In [ ]:
# UserKNN Content Based Filtering
if True:
    from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender 
    userKNNCBF = UserKNNCBFRecommender(URM_train, UCM)
    MAP_LIST = []
    tklist = [2500]
    shrinklist = [10]

    for tk in tklist:
        for sr in shrinklist:
            userKNNCBF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(userKNNCBF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        print(MAP_LIST)
        

In [ ]:
# UserKNN Collaborative Filtering
if True:
    from KNN.UserKNNCFRecommender import UserKNNCFRecommender 
    userKNNCF = UserKNNCFRecommender(URM_train)
    MAP_LIST = []
    tklist = [600]
    shrinklist = [5]

    for tk in tklist:
        for sr in shrinklist:
            userKNNCF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(userKNNCF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)    

In [ ]:
# ItemKNN Collaborative Filtering
if True:
    from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
    itemKNN = ItemKNNCFRecommender(URM_train)
    itemKNN.fit(shrink=50, topK=3)

    if eval:
        evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
        results, _ = evaluator_validation.evaluateRecommender(itemKNN)
        print('MAP: ' + str(results[10]["MAP"]))

In [ ]:
# Hybrid Recommender
from RecLib.HybridRecommender import *
if True:
    params = [(1, 0.5, 0.1, 0.4, 0.)]
    MAP_LIST = []
    for param in params:
                hybridrecommender = HybridRecommender(URM_train, userKNNCBF, itemKNN, itemKNNCBF, slim_rec, userKNNCF)
                hybridrecommender.fit(*param)
                if eval:
                    dict_scores = (evaluator_validation.evaluateRecommender(hybridrecommender))[0][10]
                    MAP_LIST.append(('alpha, beta, gamma, delta, epsilon :' + str(param), dict_scores['MAP']))
    if eval:
        sortMap(MAP_LIST)

In [ ]:
# Evaluate performance of a recommender against users with 0,1,...,itrMax interactions
rec_to_eval = hybridrecommender
itrMax = 0

if eval:
    evaluateAgainstUsers(rec_to_eval, itrMax, URM_train, URM_test)

In [ ]:
# Compare different recommenders
recommendersToCompare = [itemKNN, hybridrecommender, userKNNCF]

if eval:
    from RecLib.Evaluate import *
    compare(URM_train, URM_test, recommendersToCompare)

In [ ]:
# Predict
final_Rec = hybridrecommender

if not eval:
    output = []
    for user_id in target_users:
        output.append((user_id, final_Rec.recommend(user_id, cutoff=10)))

In [ ]:
# Writedown results

if not eval:
    import csv
    with open('submission.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["user_id", "item_list"])
        for row in output:
          ranking = ''
          for val in row[1]:
            ranking = ranking + str(val) + ' '
          writer.writerow([row[0], ranking[:-1]])